# Practicing Join Queries with Instacart Dataset

In [ ]:
Write 6 queries on dataset1 with at least 1 join per query.
● At least 2/6 queries must use an outer join.
● All queries must use a WHERE clause or ORDER BY clause or both.
● Show the output of each query in the notebook. If a query produces > 12 rows,
add a LIMIT clause to the query.
● Add a short Markdown comment above each SQL statement to describe its
function.

In [6]:
%%bigquery
select product_name, aisle_id from instacart_staging.Products

,product_name,aisle_id
0,Ground Turkey Chub,34
1,Breaded Chicken Breast Tenders,34
2,100% Pure Beef Homestyle Patties,34
3,"Chicken Breast Nuggets, Dinosaur Shaped",34
4,All Natural Premium Cornish Hen,34
...,...,...
49683,Organic Plain Skyr,100
49684,Chocolate Vanilla Malt Custard,100
49685,"8\"" Pecan Pie",100
49686,Lemon Cayenne Drinking Vinegar,100


## Query 1: This query displays what products are in what orders where the aisle is "instant foods".

In [35]:
%%bigquery
select count(*) as frequency, product_name
from instacart_staging.Products p join instacart_staging.Order_Products_Prior o on p.product_id = o.product_id
join instacart_staging.Aisles a on a.aisle_id = p.aisle_id
where a.aisle = 'instant foods'
group by product_name
order by frequency desc
limit 12

,frequency,product_name
0,10907,Organic Macaroni Shells & Real Aged Cheddar
1,8200,Macaroni Shells & White Cheddar Cheese
2,8103,Bunny Pasta with Yummy Cheese Macaroni & Cheese
3,7460,Shells & Real Aged Cheddar Macaroni & Cheese
4,6370,Organic Shells And White Cheddar
5,6204,Garlic Couscous
6,6081,Macaroni & Cheese Dinner Original Flavor
7,4890,Creamy Deluxe Shells & Real Aged Cheddar Sauce
8,4499,Spanish Rice Pilaf Mix
9,3677,Parmesan Couscous Mix


## Query 2: This query displays what each user ordered at any point, sorted by order number

In [18]:
%%bigquery
select o.user_id, p.product_name
from instacart_staging.Products p join instacart_staging.Order_Products_Prior op on p.product_id = op.product_id
join instacart_staging.Orders o on op.order_id = o.order_id
order by user_id, order_number
limit 12

,user_id,product_name
0,1,XL Pick-A-Size Paper Towel Rolls
1,1,Soda
2,1,Original Beef Jerky
3,1,Aged White Cheddar Popcorn
4,1,Organic Unsweetened Vanilla Almond Milk
5,1,Soda
6,1,Aged White Cheddar Popcorn
7,1,Bag of Organic Bananas
8,1,Cinnamon Toast Crunch
9,1,Pistachios


## Query 3: This query displays which users are most frequently purchasing baby products (aisles 82 and 92)

In [20]:
%%bigquery
select user_id, count(*) as freq
from instacart_staging.Products p join instacart_staging.Order_Products_Prior op on p.product_id = op.product_id
join instacart_staging.Orders o on o.order_id = op.order_id
where aisle_id = 92 or aisle_id = 82
group by user_id
order by freq desc
limit 12

,user_id,freq
0,128627,807
1,84092,546
2,124042,450
3,21463,427
4,8812,378
5,197502,369
6,111128,366
7,169991,366
8,108736,354
9,58919,349


## Query 4: This query determines the most popular aisles that users shop from, sorted from most to least popular

In [23]:
%%bigquery
select  count(*) as freq, aisle
from instacart_staging.Orders o join instacart_staging.Order_Products_Prior op on o.order_id = op.order_id
join instacart_staging.Products p on p.product_id = op.product_id
join instacart_staging.Aisles a on p.aisle_id = a.aisle_id
group by aisle
order by freq desc
limit 12


,freq,aisle
0,3642188,fresh fruits
1,3418021,fresh vegetables
2,1765313,packaged vegetables fruits
3,1452343,yogurt
4,979763,packaged cheese
5,891015,milk
6,841533,water seltzer sparkling water
7,722470,chips pretzels
8,638253,soy lactosefree
9,584834,bread


## Query 5: This query determines the users who purchase the most items (with no cost specified) from instacart.

In [30]:
%%bigquery
select user_id, count(*) as freq
from instacart_staging.Orders o join instacart_staging.Order_Products_Prior op on o.order_id = op.order_id
join instacart_staging.Products p on p.product_id = op.product_id
group by user_id
order by freq desc
limit 12

,user_id,freq
0,201268,3725
1,129928,3638
2,164055,3061
3,186704,2936
4,176478,2921
5,182401,2907
6,137629,2901
7,33731,2888
8,108187,2760
9,4694,2735


## Query 6: This query can help to determine how many orders have frozen items in them. This could be used to decide whether Instacart should start distributing frozen coolers to its drivers so that the company can ensure its frozen demands are being met.

In [34]:
%%bigquery
select count(distinct user_id) as freq_of_frozen_purchases
from instacart_staging.Orders o join instacart_staging.Order_Products_Prior op on o.order_id = op.order_id
join instacart_staging.Products p on p.product_id = op.product_id 
join instacart_staging.Departments d on p.department_id = d.department_id
where department = 'frozen'

,freq_of_frozen_purchases
0,163233
